In [16]:
import sys
sys.path.append('/root/semi/VFPUMC02')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from datasets.DataSet import BankDataset
from utils.DataProcessUtils import *
from semilearn import get_config, split_ssl_data, BasicDataset,get_data_loader, get_algorithm, get_net_builder,Trainer
from torchvision import transforms

In [72]:
config = {
    # 以下是可变的
    'algorithm': 'adamatch',
    'num_labels': nearest_even(5000),
    'ulb_num_labels': nearest_even(8000),
    
    # 以下是固定不变的
    'net': 'vit_tiny_patch2_32',
    'use_pretrain': True, 
    'pretrain_path': 'https://github.com/microsoft/Semi-supervised-learning/releases/download/v.0.0.0/vit_tiny_patch2_32_mlp_im_1k_32.pth',
    'include_lb_to_ulb': True,
    'batch_size': 8,
    'uratio': 2,
    'eval_batch_size':8,
    'data_dir': '/root/semi/Semi-supervised-learning/notebooks/data',
    'distributed': False,
    'gpu': 0,
    'num_classes': 2,
    'num_train_iter': 1, 
    'num_eval_iter': 1,   
    'num_log_iter': 50, 
    'epoch': 1, 
}
config = get_config(config)

/bin/sh: 1: netstat: not found


In [73]:
# create model and specify algorithm
algorithm = get_algorithm(config,  get_net_builder(config.net, from_name=False), tb_log=None, logger=None)

Files already downloaded and verified
lb count: [2500, 2500]
ulb count: [5000, 5000]
Files already downloaded and verified
unlabeled data number: 10000, labeled data number 5000
Create train and test data loaders
[!] data loader keys: dict_keys(['train_lb', 'train_ulb', 'eval'])
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
Create optimizer and scheduler
distribution alignment p_target: tensor([0.5000, 0.5000])


In [74]:
bankDataset = BankDataset()

2024-09-26 19:44:48,822 - INFO - 正在加载数据集 bank.csv
2024-09-26 19:44:49,034 - INFO - 数据集 bank.csv 加载完成，数据集形状为 (41188, 64)


In [75]:
data = expand_to_image_shape(normalize_columns(bankDataset.df.values))
data = (data * 255).astype(np.uint8)
target = bankDataset.y

In [76]:
lb_data, lb_target, ulb_data, ulb_target = split_ssl_data(config, data, target, 2,
                                                          config.num_labels,ulb_num_labels=config.ulb_num_labels,
                                                          include_lb_to_ulb=config.include_lb_to_ulb)

In [77]:
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomCrop(32, padding=int(32 * 0.125), padding_mode='reflect'),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

train_strong_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                             transforms.RandomCrop(32, padding=int(32 * 0.125), padding_mode='reflect'),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

eval_transform = transforms.Compose([transforms.Resize(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [78]:
lb_dataset = BasicDataset(config.algorithm, lb_data, lb_target, config.num_classes, train_transform, is_ulb=False)
ulb_dataset = BasicDataset(config.algorithm, lb_data, lb_target, config.num_classes, train_transform, is_ulb=True, strong_transform=train_strong_transform)
eval_dataset = BasicDataset(config.algorithm, lb_data, lb_target, config.num_classes, eval_transform, is_ulb=False)

In [79]:
# define data loaders
train_lb_loader = get_data_loader(config, lb_dataset, config.batch_size)
train_ulb_loader = get_data_loader(config, ulb_dataset, int(config.batch_size * config.uratio))
eval_loader = get_data_loader(config, eval_dataset, config.eval_batch_size)

In [80]:
# training and evaluation
trainer = Trainer(config, algorithm)
trainer.fit(train_lb_loader, train_ulb_loader, eval_loader)


Epoch: 0


/root/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2024-09-26 19:45:00,398 - INFO - confusion matrix
2024-09-26 19:45:00,399 - INFO - [[0. 1.]
 [0. 1.]]
2024-09-26 19:45:00,400 - INFO - evaluation metric
2024-09-26 19:45:00,401 - INFO - acc: 0.2500
2024-09-26 19:45:00,401 - INFO - precision: 0.1250
2024-09-26 19:45:00,401 - INFO - recall: 0.5000
2024-09-26 19:45:00,402 - INFO - f1: 0.2000
2024-09-26 19:45:00,647 - INFO - Best acc 0.2500 at epoch 0
2024-09-26 19:45:00,648 - INFO - Training finished.


model saved: ./saved_models/fixmatch/latest_model.pth
model saved: ./saved_models/fixmatch/model_best.pth


In [28]:
result = trainer.evaluate(eval_loader)

/root/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2024-09-26 18:33:24,110 - INFO - confusion matrix
2024-09-26 18:33:24,110 - INFO - [[1. 0.]
 [1. 0.]]
2024-09-26 18:33:24,111 - INFO - evaluation metric
2024-09-26 18:33:24,111 - INFO - acc: 0.6250
2024-09-26 18:33:24,112 - INFO - precision: 0.3125
2024-09-26 18:33:24,112 - INFO - recall: 0.5000
2024-09-26 18:33:24,112 - INFO - f1: 0.3846


In [31]:
result

{'acc': 0.625, 'precision': 0.3125, 'recall': 0.5, 'f1': 0.38461538461538464}